 > ## Importing libraries and data¶


In [ ]:
import pandas as pd
import numpy as np

#visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Linear Regression
from scipy.stats import linregress

#Train_test_split
from sklearn.model_selection import train_test_split

#cross_val_score
from sklearn.model_selection import cross_val_score

#scaling
from sklearn.preprocessing import StandardScaler



from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score,recall_score, confusion_matrix, classification_report,accuracy_score, f1_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score 
import matplotlib.pyplot as plt
#models

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
#score
from sklearn.metrics import r2_score

#Importing data
df = pd.read_csv("../input/insurance/insurance.csv")

> ## Analyze by describing the data

In [ ]:
# take a look at the data
df.head()

In [ ]:
# statistical measuers
display(df.describe())

In [ ]:
# Getting some informations about the dataset
df.info()

#### Findings:

There are four numerical variables:
    - Continuous:
    - Age
    - BMI
    - Charges
    - Discrete:
    - children  
There are three categorical variables:
    - Sex
    - Smoker
    - Region

##### is there is a relationship between the features and each other?

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
sns.heatmap(df.corr(), annot=True, fmt='.1g', cmap="viridis", cbar=True);

Hmmm... not much

> ## It's time to prepare our data 🙌🏻:

### checking the missing data:

In [ ]:
df.isna().sum()

No missing data, cool!

### Does it indicate that the data is really stable? check the outliers...

 first we will need some statistical information...

In [ ]:
df.describe()

In [ ]:
# Bmi
sns.boxplot(df['bmi'])

#### i got them 😈

In [ ]:
Q1 = df.bmi.quantile(0.25)
Q3 = df.bmi.quantile(0.75)
Q1, Q3

In [ ]:
IQR = Q3 - Q1
IQR

In [ ]:
lower_limit = Q1 - 1.5*IQR
upper_limit = Q3 + 1.5*IQR
lower_limit, upper_limit

#### Let's find out which this data is..

In [ ]:
df[df['bmi'] > upper_limit]

only 9 records! we can get red of them safely 

In [ ]:
df_new = df.drop(df.index[[116,401,286,543,847,860,1047,1088,1317]])

##### The rest of the features are stable, thank God .

> # DEA 

### take a look at the features and its effect in the target "charges" ...


In [ ]:
f = plt.figure(figsize=(14,6))
ax = f.add_subplot(122)
sns.scatterplot(x='bmi',y='charges',data=df,palette='viridis',hue='smoker')
ax.set_title('Scatter plot of Charges vs bmi')
plt.savefig('sc.png');

### insights:
- Most of the people who smoke are obese people
- Even though a person is not obese but he/she smokes, his/her charges tends to be higher.


In [ ]:
df["bmi"].value_counts()   # Body mass index, providing an understanding of body, ideally 18.5 to 24.9

#### I see that It will be clearer if we classify people into categories according to their "bmi" 💡

In [ ]:
df['weight_condition'] = np.nan
data = [df]

for col in data:
    col.loc[col.bmi < 18.5, "weight_condition"] = "Underweight"
    col.loc[(col.bmi >= 18.5) & (col.bmi < 24.986), "weight_condition"] = "Normal"
    col.loc[(col.bmi >= 25) & (col.bmi < 29.926), "weight_condition"] = "Overweight"
    col.loc[col.bmi >= 30, "weight_condition"] = "Obese"


In [ ]:
df.drop("bmi", axis = 1, inplace = True)
display(df.head())

### age VS charges

In [ ]:
f = plt.figure(figsize=(14,6))
ax = f.add_subplot(121)
sns.scatterplot(x='age',y='charges',data=df,palette='magma',hue='smoker',ax=ax)
ax.set_title('Scatter plot of Charges vs age')


#### Insights
- Charges increases as the age of the person increases
- Smokers and obese people have the highest charges regardless of their age.


#### i'll do the same thing with "age" so that the data to be follow the same flo 🤙🏻

In [ ]:
df['age_cat'] = np.nan
lst = [df]

for col in lst:
    col.loc[(col['age'] >= 18) & (col['age'] <= 35), 'age_cat'] = 'Young Adult'
    col.loc[(col['age'] > 35) & (col['age'] <= 55), 'age_cat'] = 'Senior Adult'
    col.loc[col['age'] > 55, 'age_cat'] = 'Elder'
    

In [ ]:
df.drop("age", axis = 1, inplace = True)
df.head()

### Relation between sex and charges: 

In [ ]:
plt.style.use("seaborn")
fig, ax = plt.subplots(figsize=(5,5))
plt.pie(x=df["sex"].value_counts(), 
        colors=["skyblue","pink"], 
        labels=["Male","Female"], 
        shadow = True, 
        autopct="%1.2f%%", 
        explode = (0, 0.1)
        )
plt.show()

##### Almost the same proportion...

In [ ]:
#Cheking the charges distributions for males and females
x1 = sns.FacetGrid(df, row='sex', height=4, aspect=3)
x1 = x1.map(sns.distplot, 'charges', color='cornflowerblue')
plt.show()


##### As we can see the two distributions are almost the same for both women/men, so we can affirm that there is no influence on the medical charges when it comes to the sex variable.

###  let's see how many children our patients have.



In [ ]:
plt.style.use("seaborn")
fig, ax = plt.subplots(figsize=(6,6))
sns.countplot(df["children"], palette="hls");

##### Most patients do not have children

In [ ]:
#Creating a violinplot for each category
plt.figure(figsize=(12,4))
sns.violinplot(x='children', y='charges', data=df, hue='sex', palette='YlGnBu')
plt.title('Charges according to number of children', size='23')
plt.ylabel('Charges',size=18)
plt.xlabel('Number of children',size=18)
plt.show()

As we can see, almost all categories have the same range and mean of costs also the distributions are very similar, except for the people who have 5 children. This might be because of the small size of the sample of this kind of people!

### Relation between smoking and charges: 

In [ ]:
#Countplot to compare the number of smokers and non-smokers
plt.figure(figsize=(12,4))
sns.countplot(x='smoker', data=df, hue='sex', palette='YlOrBr') 
plt.title('Number of smokers and non-smokers', size='23')
plt.ylabel('Count',size=18)
plt.xlabel('Smoker',size=18)
plt.show()

#### notice :
- 0 for females
- 1 for males
------------------
- smokers = 1: yes
- smokers = 0: no

 Males tend to smoke more than females.

##### As we can see charges for smokers are much higher than charges for non-smokers. Sex doesn't have any effect on charges when you are a smoker

###  Relation between regions and charges: 

In [ ]:
#Countplot to compare the number of individuals from diffrent regions
plt.figure(figsize=(12,4))
sns.countplot(x='region', data=df, palette='husl') 
plt.title('Number of individuals from diffrent regions', size='23')
plt.ylabel('Count',size=18)
plt.xlabel('Region',size=18)
plt.show()

##### Almsot all the regions have equal number of individuals.

## Conclusion:
##### We have found out that region and gender does not bring significant difference on charges among its groups. Age, BMI,      number of children and smoking are the once that drives the charges. 
As:
-  age has an impact on the charges, when a person is older the health costs are larger.
-  if you are a smoker you must expect some huge medical charges compared to non-smokers. Especially for people who have high BMI values (>35) it will result very serious health care charges.
- no matter where you live, this won't have any impact on your medical insurance bills.
- the number of children doesn't affect the medical costs billed by health insurance.
-  it doesn't matter if you are a men or a women your health bills won't change.

### before go  to modeling, we have to prepare the data types of the features:

> # label encoding:


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
les = {}
for col in ['sex','smoker','region', 'weight_condition', 'age_cat']:
    les[col] = LabelEncoder()
    df[col]  = les[col].fit_transform(df[col])

In [ ]:
df.head()

In [ ]:
df =df.drop(columns=['sex','region'])
df.head()

##### Great ♥️

### We are almost ready to finally prepare the models! so let's split the data to work on it...

> # Modeling :
There are good models, But we looking for the Best!

> # Splitting the data into Training data & Testing Data:

In [ ]:
x= df.drop('charges',axis=1).values
y =df.charges.values
x_tr,x_tst,y_tr,y_tst = train_test_split(x,y,test_size=.2,random_state=25)

- Feature Scaling

In [ ]:
sc = StandardScaler()
x_tr = sc.fit_transform(x_tr)
x_tst = sc.transform(x_tst)

- Experience 8 algorithms


In [ ]:
lr  = LinearRegression()
knn = KNeighborsRegressor(n_neighbors=10)
dt  = DecisionTreeRegressor(max_depth = 3)
rf  = RandomForestRegressor(max_depth = 3, n_estimators=500)
ada = AdaBoostRegressor( n_estimators=50, learning_rate =.01)
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=100, learning_rate =.2)
xgb = XGBRegressor(max_depth = 3, n_estimators=50, learning_rate =.2)
cb  = CatBoostRegressor(learning_rate =.01, max_depth =5, verbose = 0)

regressors = [('Linear Regression', lr), ('K Nearest Neighbours', knn),('Decision Tree', dt), ('Random Forest', rf), 
              ('AdaBoost', ada),('Gradient Boosting Regressor', gbr),('XGBRegressor', xgb),('CatBoostRegressor', cb)]


In [ ]:

for regressor_name, regressor in regressors:

    regressor.fit(x_tr, y_tr)    
  
    y_pred = regressor.predict(x_tst)
    accuracy = round(r2_score(y_tst,y_pred),3)*100
    
    print('{:s} : {:.0f} %'.format(regressor_name, accuracy))
    plt.rcParams["figure.figsize"] = (20,8)
    plt.bar(regressor_name,accuracy)

### Best Models
- Gradient Boosting Regressor : 88 %
- CatBoostRegressor : 88 %
- XGBRegressor : 88 %


 ### R square (r2) Score & Root Mean Squared Error(RMSE)
 

- Gradient Boosting Regressor

In [ ]:
y_predict = gbr.predict(x_tst)
gbr_r2_score = r2_score(y_tst, y_predict)
print('R square Score = ', round(gbr_r2_score, 5))

mse = mean_squared_error(y_tst, y_predict)
rmse_1 = np.sqrt(mse)
print('Root Mean Squared Error = ', round(rmse_1,5))

- XGBRegressor
- note : the best algorithm

In [ ]:
y_predict = xgb.predict(x_tst)
xgb_r2_score = r2_score(y_tst, y_predict)
print('R square Score = ', round(xgb_r2_score, 5))

mse = mean_squared_error(y_tst, y_predict)
rmse_2 = np.sqrt(mse)
print('Root Mean Squared Error = ', round(rmse_2, 5))

- CatBoostRegressor.


In [ ]:
y_predict = cb.predict(x_tst)
cb_r2_score = r2_score(y_tst, y_predict)
print('R square Score = ', round(cb_r2_score, 5))

mse = mean_squared_error(y_tst, y_predict)
rmse_3 = np.sqrt(mse)
print('Root Mean Squared Error = ', round(rmse_3, 5))

### Cross Validation
It is a resampling procedure wich is used to evaluate the machine learning models on limited data samples. Its goal is to predict new data that is that is not tested before. 

In [ ]:
score = cross_val_score(xgb, x, y, cv=5)
print(score)

### Model Accuracy Comparison

In [ ]:

r2_scores = [gbr_r2_score*1000,xgb_r2_score*1000,cb_r2_score*1000]
model_names = ["Gradient Boosting R","XGBRegressor","CatBoostRegresso"]

total_bar = np.arange(len(model_names))
color = ['#9edd1d', '#3edd1d', '#f7c851']

fig, ax = plt.subplots(figsize=(15, 5))
bar = plt.bar(model_names, r2_scores, alpha=1,color=color)
plt.xticks(total_bar, model_names)
plt.ylabel('Accuracy',fontsize=15, color='black')
plt.xlabel('Model Name',fontsize=15, color='black')
plt.title('Model Performance Comparison', fontsize=16, color='black', fontweight='bold')

def autolabel(bar):
    for bar in bar:
        height = int(bar.get_height())
        ax.text(bar.get_x() + .4, .5*height,
                height, va='bottom',
                fontsize=14, color='black')
        
autolabel(bar)

plt.show()

In [ ]:
m = [rmse_1,rmse_2,rmse_3]
model_names = ["Gradient Boosting R","XGBRegressor","CatBoostRegresso"]

total_bar = np.arange(len(model_names))
color = ['#9edd1d', '#3edd1d', '#f7c851']

fig, ax = plt.subplots(figsize=(15, 5))
bar = plt.bar(model_names, m, alpha=1,color=color)
plt.xticks(total_bar, model_names)
plt.ylabel('Accuracy',fontsize=15, color='black')
plt.xlabel('Model Name',fontsize=15, color='black')
plt.title('Root Mean Squared Error', fontsize=16, color='black', fontweight='bold')

def autolabel(bar):
    for bar in bar:
        height = int(bar.get_height())
        ax.text(bar.get_x() + .4, .5*height,
                height, va='bottom',
                fontsize=14, color='black')
        
autolabel(bar)

plt.show()